In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter
import sounddevice as sd

/home/huy/Project/ELSSA/elssa/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Download clone voice

In [ ]:
# !!yt-dlp -x --audio-format mp3 -o "../openvoice/resources/ref_voice.mp3" "https://www.youtube.com/shorts/LZIxb2VTdAg"

['[youtube] Extracting URL: https://www.youtube.com/shorts/LZIxb2VTdAg',
 '[youtube] LZIxb2VTdAg: Downloading webpage',
 '[youtube] LZIxb2VTdAg: Downloading tv client config',
 '[youtube] LZIxb2VTdAg: Downloading tv player API JSON',
 '[youtube] LZIxb2VTdAg: Downloading ios player API JSON',
 '[youtube] LZIxb2VTdAg: Downloading m3u8 information',
 '[info] LZIxb2VTdAg: Downloading 1 format(s): 251-2',
 '[download] Destination: ../openvoice/resources/ref_voice.webm',
 '',
 '[download]   0.2% of  415.03KiB at  Unknown B/s ETA Unknown',
 '[download]   0.7% of  415.03KiB at  Unknown B/s ETA Unknown',
 '[download]   1.7% of  415.03KiB at    5.19MiB/s ETA 00:00  ',
 '[download]   3.6% of  415.03KiB at    8.53MiB/s ETA 00:00',
 '[download]   7.5% of  415.03KiB at   14.17MiB/s ETA 00:00',
 '[download]  15.2% of  415.03KiB at   11.73MiB/s ETA 00:00',
 '[download]  30.6% of  415.03KiB at   12.56MiB/s ETA 00:00',
 '[download]  61.4% of  415.03KiB at   14.82MiB/s ETA 00:00',
 '[download] 100.0% of 

In [ ]:
# !!yt-dlp -x --audio-format wav -o "../openvoice/outputs/raw_speech.wav" "https://www.youtube.com/shorts/ul5VV3uWUU4"

['[youtube] Extracting URL: https://www.youtube.com/shorts/ul5VV3uWUU4',
 '[youtube] ul5VV3uWUU4: Downloading webpage',
 '[youtube] ul5VV3uWUU4: Downloading tv client config',
 '[youtube] ul5VV3uWUU4: Downloading player 8e20cb06-main',
 '[youtube] ul5VV3uWUU4: Downloading tv player API JSON',
 '[youtube] ul5VV3uWUU4: Downloading ios player API JSON',
 '[youtube] ul5VV3uWUU4: Downloading m3u8 information',
 '[info] ul5VV3uWUU4: Downloading 1 format(s): 251',
 '[download] Destination: ../openvoice/outputs/raw_speech.webm',
 '',
 '[download]   0.2% of  531.36KiB at  Unknown B/s ETA Unknown',
 '[download]   0.6% of  531.36KiB at    1.76MiB/s ETA 00:00  ',
 '[download]   1.3% of  531.36KiB at    3.35MiB/s ETA 00:00',
 '[download]   2.8% of  531.36KiB at    6.16MiB/s ETA 00:00',
 '[download]   5.8% of  531.36KiB at   11.15MiB/s ETA 00:00',
 '[download]  11.9% of  531.36KiB at   11.48MiB/s ETA 00:00',
 '[download]  23.9% of  531.36KiB at   14.71MiB/s ETA 00:00',
 '[download]  48.0% of  531.36

In [2]:
ckpt_base = '../models/openvoice/checkpoints/base_speakers/EN'
ckpt_converter = '../models/openvoice/checkpoints/converter'
reference_speaker = '../assets/audio/ref_voice.mp3'
output_dir = '../models/openvoice/outputs'
target_dir = '../models/openvoice/processed'
device="cuda:0" if torch.cuda.is_available() else "cpu"

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')


source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

/home/huy/Project/ELSSA/elssa/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
import json
with open(f'{ckpt_base}/config.json', 'r') as f:
    config = json.load(f)

In [8]:
# Run the base speaker tts
text = input("Enter the text to synthesize: ")
src_path = f'{output_dir}/raw_speech.wav'
raw_audio = base_speaker_tts.tts(text, speaker='default', language='English', speed=1.0)

haɪ.
 length:4
 length:4


In [9]:
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir=target_dir, vad=True)

In [10]:

# Run the tone color converter
encode_message = "@MyShell"
speech = tone_color_converter.convert(
    raw_audio=raw_audio,
    src_se=source_se, 
    tgt_se=target_se, 
    message=encode_message)


# Giả sử `audio_np` là mảng âm thanh numpy 1D hoặc 2D (mono/stereo)
sd.play(speech, samplerate=config['data']['sampling_rate'])  # Phát âm thanh
sd.wait()  # Đợi cho phát xong rồi mới chạy tiếp